In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import math
from itertools import zip_longest

El objetivo de este proyecto es recopilar un histórico de datos de mediciones de calidad del aire en Andalucía. Para construir este histórico, se partirá de los informes generados por la Junta de Andalucía. Los informes se presentan de forma diaria. La Url se modifica para mostrar los datos de un día concreto.

In [3]:
URL = 'http://www.juntadeandalucia.es/medioambiente/atmosfera/informes_siva/mar21/nse210315.htm'
page = requests.get(URL)

#Función creada para reorganizar la lista de medidas dividiéndola en bloques en función de la longitud de su cabecera
def grouper(n, it_list, padValue=None):
    """ n -> nºcolumnas por fila
        it_list -> lista sobre la que iterar
        padValue -> valor por defecto
    """
    return zip_longest(*[iter(it_list)]*n, fillvalue=padValue)

status_code = page.status_code
# Comprobamos que la petición nos devuelve un Status Code = 200
if status_code == 200:
    soup = BeautifulSoup(page.content, 'html.parser')

Para alcanzar el objetivo, se creará un dataframe que recopile primero los datos de la estación y localización y posteriormente los datos de las mediciones. El principal problema, es que las tablas presentes en el documento no tienen identificador, y no todas las tablas tienen información numérica de mediciones, algunas tablas son solo cabeceras con la localización de las medidas tomadas en la tabla siguiente. 

Se creará en primer lugar una tabla paramétrica con todos los datos de localización presentes en el informe para un día concreto.

In [4]:
tables = soup.findAll("table")
l_province = list()
l_municipio = list()
l_station = list()
l_dir = list()

Con lo anterior tenemos una lista de tablas de con la información de localización, necesitamos recopilar esta información en una vector para insertarlo en la fila de cada tabla.

In [5]:
#Iteramos por todas las tablas:
#Índice para iterar por las filas del dataframe de localización
j = 0
#Inicialización del esquema del dataframe final
df_final = pd.DataFrame({ 'FECHA-HORA': [],'SO2': [],'PART': [], 'NO2': [], 'CO': [], 'O3': [], 'SH2': [],
                         'Provincia': [], 'Municipio': [], 'Estacion': [], 'Direccion': []})
for table in tables:
    tabla_measures = list()
    #Se ha utilizado la palabra clave provincia para filtrar
    if table.text.startswith('Provincia'):
        indice_prov = table.text.index('Provincia')
        indice_mun = table.text.index('Municipio')
        indice_est = table.text.index('Estacion')
        indice_dir = table.text.index('Direccion')
        l_province.append(table.text[indice_prov+9:indice_mun])
        l_municipio.append(table.text[indice_mun+9:indice_est])
        l_station.append(table.text[indice_est+8:indice_dir])
        l_dir.append(table.text[indice_dir+9:])

        #Variable de control que indica que tabla se está recorriendo en dicha iteración, de localización (1) o de mediciones (0)
        #Esto permite ir incrementando el índice para la tabla paramétrica de provincias
        ind_tabla_prov = 1

    elif not table.text.startswith('RED DE VIGILANCIA'):
        l_cab = list()
        table_Cabecera = table.find_all('td', {'class': 'CabTabla'})
        for cab in table_Cabecera:
            l_cab.append(cab.text)

        measures_tr = table.find_all('tr')
        measures = list()
        for tr in measures_tr:
            measures_td = tr.find_all('td')
            for td in measures_td:
                if td.text not in ('FECHA-HORA', 'SO2', 'PART', 'NO2', 'CO', 'O3', 'SH2') and not td.text.startswith('\nNota:'):
                    measures.append(td.text)
        
        if len(l_cab) != 0:
            tabla_measures = list([l_cab])
            tabla_measures+=list(grouper(len(l_cab), measures, math.ceil(len(measures)/len(l_cab))))
            df_measures = pd.DataFrame(tabla_measures[1:],columns=tabla_measures[0])
            
        df_loc = pd.DataFrame({'Provincia': l_province, 'Municipio': l_municipio, 'Estacion': l_station, 'Direccion': l_dir})

        #Si el índice de la lista no ha llegado a 12 (nºtablas de la URL, habrá que cambiarlo para generalizar) o no es la primera
        #tabla de provincia, se crea un df intermedio con la posición de la paramétrica correspondiente. Se replica la fila
        #recogida tantas veces como filas tenga el dataframe de medidas intermedio del paso anterior
        if l_province and j < 12:
            df_loc_iter = pd.DataFrame(np.repeat(pd.DataFrame({'Provincia': l_province[j],
                                    'Municipio': l_municipio[j],
                                    'Estacion': l_station[j],
                                    'Direccion': l_dir[j]},index=[0]).values,len(df_measures),axis=0),columns = ['Provincia','Municipio',
                                                                                                       'Estacion','Direccion'])
            #Se concatenan los dos df intermedios de medidas y localizaciones para la presente iteración
            #Se añaden las filas al dataframe final con toda la información
            df_iter = pd.concat((df_measures,df_loc_iter), axis = 1)
            df_final = df_final.append(df_iter,sort=True)
            #Si se ha recorrido tabla de provincia en el paso anterior, se incrementa +1 el indicador
            if ind_tabla_prov == 1:
                j = j + 1
            #Variable de control de tabla de provincia. Dado que se ha recorrido una tabla de medidas, se pasa el indicador a 0
            ind_tabla_prov = 0
    
    else:
        print("Status code %d" % status_code)
    

In [6]:
df_final = df_final[['Provincia', 'Municipio',
                     'Direccion', 'Estacion','FECHA-HORA','CO','NO2','O3','PART','SO2','SH2']].reset_index()
df_final.head()

,index,Provincia,Municipio,Direccion,Estacion,FECHA-HORA,CO,NO2,O3,PART,SO2,SH2
0,0,SEVILLA,SEVILLA,Avd. ANDALUCIA,RANILLA,15/03/21-00:10,897,17,NaN,NaN,4,NaN
1,1,SEVILLA,SEVILLA,Avd. ANDALUCIA,RANILLA,15/03/21-00:20,877,12,NaN,NaN,4,NaN
2,2,SEVILLA,SEVILLA,Avd. ANDALUCIA,RANILLA,15/03/21-00:30,871,11,NaN,NaN,4,NaN
3,3,SEVILLA,SEVILLA,Avd. ANDALUCIA,RANILLA,15/03/21-00:40,886,11,NaN,NaN,4,NaN
4,4,SEVILLA,SEVILLA,Avd. ANDALUCIA,RANILLA,15/03/21-00:50,905,12,NaN,NaN,4,NaN


In [17]:
df_final.to_csv('AirQualityAndalusia.csv', index = False, header=True,encoding='utf-8-sig',sep=';')